In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os

from skimage.io import imread
from skimage.transform import resize

import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten

from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions

<h1>Neural Networks Architectures - Transfer Learning</h1>

In [3]:
vgg19 = VGG19(include_top = False, input_shape = (128, 128, 3))

In [4]:
BASE_DIR = os.path.abspath(os.path.join("E:\Документи\СофтУни\python_deep_learning\image_related_neural_networks\cats_and_dogs", "training_set"))
CATS_DIR = os.path.join(BASE_DIR, "cats")
DOGS_DIR = os.path.join(BASE_DIR, "dogs")

BATCH_SIZE = 16
IMAGE_SIZE = (128, 128)

cat_images = [os.path.join(CATS_DIR, f) for f in os.listdir(CATS_DIR)]
dog_images = [os.path.join(DOGS_DIR, f) for f in os.listdir(DOGS_DIR)]

all_images = cat_images + dog_images

def preprocess_images(filename):
    file = tf.io.read_file(filename)
    
    img = tf.image.decode_jpeg(file)
    img = tf.image.resize(img, IMAGE_SIZE)
    img = img / 255.0
    
    label = tf.strings.split(filename, sep = "\\")[-2] == "cats"
    label = tf.cast(label, tf.int32)
    
    return img, label

dataset = tf.data.Dataset.from_tensor_slices(all_images) \
    .shuffle(len(all_images)) \
    .map(preprocess_images) \
    .batch(BATCH_SIZE) \
    .prefetch(tf.data.AUTOTUNE) \
    .repeat()

In [5]:
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [6]:
classifier = Flatten()(vgg19.output)
dense1 = Dense(64, activation = "relu")(classifier)
dense2 = Dense(32, activation = "relu")(dense1)
dense3 = Dense(1, activation = "sigmoid")(dense2)

In [7]:
model = Model(inputs = vgg19.input, outputs = dense3)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [9]:
# We exclude already trained layers and keep for training only last added layers by us (Flatten and Dense).
for layer in vgg19.layers:
    layer.trainable = False

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [11]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [12]:
model.fit(dataset, epochs = 1, steps_per_epoch = 500)

500/500 [==============================] - 82s 148ms/step - loss: 0.3704 - accuracy: 0.8299
